# Analysis 1- Shreyas Bhatt

## Milestone 2
#### Task 2 : Load Dataset

In [ ]:
# loading dataframe
import pandas as pd
pd.read_csv("../data/raw/FT1000.csv")

,Rank,Name,Ranked2021,Ranked2020,Country,Sector,CAGR,Revenue2020,Revenue2017,Employees2020,Employees2017,FoundingYear
0,1,Swappie,No,No,Finland,Technology,477.43,97611814,507000,218,1,2016
1,2,Kilo Health,No,No,Lithuania,Health,450.05,57318766,344428,177,10,2013
2,3,OCI,No,No,UK,Financial Services,409.59,568322073,4325512,32,4,2012
3,4,OnlyFans,No,No,UK,Technology,393.63,316732986,2652185,800,4,2016
4,5,Enpal,No,No,Germany,Energy,386.88,56109613,486165,365,9,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,peopleForecast,No,No,Germany,Technology,36.59,2086411,818658,45,15,2014
996,997,Digitalpa,No,No,Italy,Management Consulting,36.59,1731340,679440,12,41,2014
997,998,Faktenkontor,No,No,Germany,Advertising,36.59,30967000,12153143,150,100,2003
998,999,CLAREO,Yes,Yes,France,Retail,36.58,18854708,7399729,53,13,2012


#### Task 3 : Define and refine your research questions

**To what extent does the revenue of a company impact it's growth rate and other company dynamics?**

It is quite common for many companies to experience an unprecedented growth during their early stages of operation. Although, it is also apparant that this growth is temporary and that once a company gains enough traction, growth generally slows down. I wish to understand and quantify the nature of this relationship by comparing the revenue of the company with CAGR (compounded growth). I also wish to understand what sorts of metrics seem to be helpful for large companies to maintain growth- from employee count to where the company is based and so on. 

My dataset features revenue entries for 2020 and for 2017, and moving on I may have to either combine this data in someway or observe the relationships of both of these revenues. Currently I feel that it may incorporate some inconsistency if I merge them, as well as looking at 2020 solely would likely not be a fair representation of the companies given the economic climate.